# Tutorial B5 (Automated CPA Attack With Analyzer)

This tutorial will take you through a complete attack on a software AES implementation. The specific implementation being attacked is a well-known AES implementation written in C, which is likely to be similar to other implementations used by proprietary systems.

## Capturing Power Traces

### Setup

We'll use some helper scripts to make setup and programming easier. If you're using an XMEGA or STM (CWLITEARM) target, binaries with the correct should be setup for you:

In [ ]:
%run "Helper Scripts/CWLite_Connect.ipynb"

In [ ]:
%run "Helper Scripts/Setup_Target_Generic.ipynb"

In [ ]:
# uncomment based on your target
#%run "Helper Scripts/Program_XMEGA.ipynb"
%run "Helper Scripts/Program_STM.ipynb"
#%run "Helper Scripts/No_Programmer.ipynb"
fw_path = "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-cwlitearm.hex"

In [ ]:
# program the target
program_target(scope, fw_path)

In addition, before we capture our traces, we'll need to create a ChipWhipserer project, since that's what Analyzer expects for an input:

In [ ]:
from chipwhisperer.common.api.ProjectFormat import ProjectFormat
project = ProjectFormat()
project.setFilename("jupyter_test.cwp")

And we can get the class used to hold our traces by:

In [ ]:
from datetime import datetime
import copy

tc = copy.copy(project.getTraceFormat())
starttime = datetime.now()
prefix = starttime.strftime('%Y.%m.%d-%H.%M.%S') + "_"
tc.config.setConfigFilename(project.datadirectory + "traces/config_" + prefix + ".cfg")
tc.config.setAttr("prefix", prefix)
tc.config.setAttr("date", starttime.strftime('%Y-%m-%d %H:%M:%S'))

### Capturing Traces

Below you can see the capture loop. The main body of the loop loads some new plaintext, arms the scope, sends the key and plaintext, then finally records and our new trace into our trace class. We'll also keep track of our keys manually for checking our answer later.

In [ ]:
#Capture Traces
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import numpy as np
import time

ktp = AcqKeyTextPattern_Basic(target=target)

keys = []
N = 50  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    keys.append(key)

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    _ = target.readOutput()  # clears the response from the serial port
    #traces.append(scope.getLastTrace())
    tc.addTrace(scope.getLastTrace(), text, "", key)
    
#Calling closeAll() to ensure flush to disk happens
tc.closeAll()

Now that we have our traces, we need to tell the project that the traces are loaded and add them to the project's trace manager.

In [ ]:
project.traceManager().appendSegment(tc)

#Save project file
project.save()

We're now done with the ChipWhisperer hardware, so we should disconnect from the scope and target:

In [ ]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

## Analysis

Now that we have our traces, we can begin our attack! We'll start off by importing everything we need for the attack:

In [ ]:
from chipwhisperer.analyzer.attacks.cpa import CPA
from chipwhisperer.analyzer.attacks.cpa_algorithms.progressive import CPAProgressive
from chipwhisperer.analyzer.attacks.models.AES128_8bit import AES128_8bit, SBox_output
from chipwhisperer.analyzer.preprocessing.add_noise_random import AddNoiseRandom
from chipwhisperer.common.results.outputvstime import OutputVsTimeNoGUI

Next, we'll add our traces to a preprocessing module. We can feed `project.traceManager()` right into `attack.setTraceSource()`, but we could also add pre-processing inbetween (more about this later). We'll also re-open the traces, in this case it is required since the call to `closeAll()` would have flushed the buffers.

In [ ]:
#Force reload of project data (if you comment out 'closeAll()' this isn't needed)

#We also rebuild the project object in case you only want to run this half
from chipwhisperer.common.api.ProjectFormat import ProjectFormat
project = ProjectFormat()

project.load('./jupyter_test.cwp')

And then we can setup our attack:

In [ ]:
attack = CPA()
N = 50 #number of traces

tm = project.traceManager()

leak_model = AES128_8bit(SBox_output)
attack.setAnalysisAlgorithm(CPAProgressive, leak_model)
attack.setTraceSource(tm)
attack.setTraceStart(0)
attack.setTracesPerAttack(N)
attack.setIterations(1)
attack.setReportingInterval(10)
attack.setTargetSubkeys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
attack.setPointRange((0, -1))

And then actually run it:

In [ ]:
def stats_ready():
    print("Iterating...")
attack_results = attack.processTracesNoGUI(stats_ready)
print("Done.")

Once you see the above block complete, all the heavylifting is done! All that's left is to actually look at the data. Everything important is contained in the `attack_results` class that `attack.processTracesNoGUI()` returned. Note we haven't put anything interesting in the stats_ready(), we'll use that later. For now let's look at the class type:

In [ ]:
print(attack_results.__class__)

We can find the max correlation for every one of the subkey by calling `attack_results.findMaximums()`, which returns a list of the subkeys, the point location of the max if calculated, and the correlation (which is a value between 0 and 1 that effectively tells us how well our guess fit the data).

Note the "point location of the max" is normally not calculated/tracked, and thus returns as a 0. Using the pandas library lets us print them nicely in a DataFrame. We have to transpose the frame to get our expected orientation:

In [ ]:
stat_data = attack_results.findMaximums()

#Print all guesses
import pandas as pd
df = pd.DataFrame(stat_data).transpose()

print(df.head())

Even better, we can use the `.style` method to customize this further. This also lets us pass a formatting function, which for example will remove the extra '0' and make the datatype look something more like we expect.

In [ ]:
#We might want to print guesses in hex and limit decimals (looks more natural)
def format_stat(stat):
    return str("{:02X}<br>{:.3f}".format(stat[0], stat[2]))

df.head(10).style.format(format_stat)

You can do lots of formatting thanks to the pandas library! Check out https://pandas.pydata.org/pandas-docs/stable/style.html for more details.

It's up to us to print and interpret the data. Below, you can find a simple loop to print out our best guess for the key, the actual key, and the correlation. With any luck, you should see a bunch of numbers printed which match the ones in parentheses and some text at the bottom that says we got the encryption key.

In [ ]:
key = keys[0]
i = 0
all_right = True
for bnum in stat_data:
    print("Best Guess (Actual) = 0x{:02X} (0x{:02X}), Corr = {}".format(bnum[0][0], key[i], bnum[0][2]))
    if bnum[0][0] != key[i]:
        all_right = False
    i += 1
    
if all_right:
    print("We guessed the encryption key!")
else:
    print("We didn't guess the encryption key")

That's all well and good, but how good much better was our guess than the other ones? To show that, we can use ChipWhisperer's `OutputVsTimeNoGUI`:

In [ ]:
outvstime = OutputVsTimeNoGUI(stats, key)
bnum = 0
ret = outvstime.getPlotData(bnum)
xrange = ret[0]

And then plot the data:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()
p.line(xrange, ret[2], line_color='green')
p.line(xrange, ret[3], line_color='green')

p.line(xrange, ret[1], line_color='red')
show(p)

You should see a graph of red and green in time (samples). In red is the correlation of the correct subkey for the first byte, while the rest are in green.

You should see two or three distinctive red spikes. The first is the spot where the sbox lookup for the subkey we guessed actually happens (the later ones are from later steps in the AES operation).

What about the rest of the bytes in the key? We can get and plot that easily as well:

In [ ]:
rets = []
for bnum in range(0, 16):
    rets.append(outvstime.getPlotData(bnum))

p = figure()
for ret in rets:
    p.line(xrange, ret[2], line_color='green')
    p.line(xrange, ret[3], line_color='green')
    
for ret in rets:
    p.line(xrange, ret[1], line_color='red')

show(p)

Finally - how would be get real-time feedback? The following example code gives us a table that updates in real-time so you have an idea of the status.

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

def format_stat(stat):
    if isinstance(stat, (np.void,list)):
        return str("{:02X}<br>{:.3f}".format(stat[0], stat[2]))
    else:
        return str(stat)

def stats_ready():
    stats = attack.getStatistics()
    stats.setKnownkey(key)
    stat_data = stats.findMaximums()
    df = pd.DataFrame(stat_data).transpose()

    #Add PGE as row too
    df_pge = pd.DataFrame(stats.pge).transpose().rename(index={0:"PGE="}, columns=int)
    
    df = pd.concat([df_pge, df], ignore_index=False)
    
    clear_output(wait=True)
    display(df.head(10).style.format(format_stat))
    
attack_results = attack.processTracesNoGUI(stats_ready)

## Conclusion

You should now have completed a successful CPA attack! 

You can move onto more advanced tutorials, especially showing you how the actual attack works when performed manually (Tutorial B6). This tutorial also utilized tiny-AES128-C for Arm targets, which uses the same operations as the XMEGA target. A later tutorial will preform this attack on a more typical 32 bit AES implementation.